In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# !pip install pandas plotly dash

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# import plotly.express as px
# from dash import Dash, dcc, html, Input, Output


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Read the CSV file into a DataFrame
df = pd.read_csv("/kaggle/input/dallas-public-safety/Police_Incidents.csv")

# Display the DataFrame
# print(df["Service Number ID"])


In [ ]:
# # Group the data and calculate incident counts
# grouped = df.groupby(['Sector', 'Division', 'Day1 of the Week', 'Time1 of Occurrence']).size().reset_index(name='IncidentCount')

# # Define a scoring function (you can customize this based on your preference)
# def calculate_safety_score(incident_count):
#     return 100 / (incident_count + 1)  # Example scoring function (higher count -> lower score)

# # Calculate safety scores based on incident counts
# grouped['SafetyScore'] = grouped['IncidentCount'].apply(calculate_safety_score)

# # Merge safety scores back into the original dataset
# df = pd.merge(df, grouped[['Sector', 'Division', 'Day1 of the Week', 'Time1 of Occurrence', 'SafetyScore']], on=['Sector', 'Division', 'Day1 of the Week', 'Time1 of Occurrence'], how='left')

# # Display the updated DataFrame with the 'SafetyScore' column
# print(df['SafetyScore'])

In [ ]:
# Take the first 1000 entries as a subset
subset_df = df.head(10000)

# Calculate the total incident count for each Division
division_counts = subset_df.groupby('Division').size().reset_index(name='IncidentCount')

# Convert the division_counts DataFrame to a dictionary
division_incident_dict = dict(zip(division_counts['Division'], division_counts['IncidentCount']))

print(division_incident_dict)

In [ ]:
# Take the first 1000 entries as a subset
subset_df_2 = df.head(10000)

# Group the data by Division and Sector, and calculate incident counts
grouped = subset_df_2.groupby(['Division', 'Sector']).size().reset_index(name='IncidentCount')

# Create a nested dictionary to store the data
division_sector_dict = {}

# Iterate through the rows of the grouped DataFrame to populate the nested dictionary
for _, row in grouped.iterrows():
    division = row['Division']
    sector = row['Sector']
    incident_count = row['IncidentCount']
    
    # If the Division is not in the dictionary, add it
    if division not in division_sector_dict:
        division_sector_dict[division] = {}
    
    # Add the Sector and its corresponding incident count to the Division's dictionary
    division_sector_dict[division][sector] = incident_count

print(division_sector_dict)

In [ ]:
# Take the first 1000 entries as a subset
subset_df_3 = df.head(1000)

# Group the data by Division, Sector, and Day of the Week, and calculate incident counts
grouped = subset_df_3.groupby(['Division', 'Sector', 'Day1 of the Week']).size().reset_index(name='IncidentCount')

# Create a nested dictionary to store the data
division_sector_day_dict = {}

# Iterate through the rows of the grouped DataFrame to populate the nested dictionary
for _, row in grouped.iterrows():
    division = row['Division']
    sector = row['Sector']
    day = row['Day1 of the Week']
    incident_count = row['IncidentCount']
    
    # If the Division is not in the dictionary, add it
    if division not in division_sector_day_dict:
        division_sector_day_dict[division] = {}
    
    # If the Sector is not in the Division's dictionary, add it
    if sector not in division_sector_day_dict[division]:
        division_sector_day_dict[division][sector] = {}
    
    # Add the Day and its corresponding incident count to the Sector's dictionary
    division_sector_day_dict[division][sector][day] = incident_count

print(division_sector_day_dict)


In [ ]:
# Take the first 1000 entries as a subset
subset_df_4 = df.head(1000)

# Group the data by Division, Sector, Day of the Week, Time of Occurrence, and calculate incident counts
grouped = subset_df_4.groupby(['Division', 'Sector', 'Day1 of the Week', 'Time1 of Occurrence']).size().reset_index(name='IncidentCount')

# Create a nested dictionary to store the data
division_sector_day_time_dict = {}

# Iterate through the rows of the grouped DataFrame to populate the nested dictionary
for _, row in grouped.iterrows():
    division = row['Division']
    sector = row['Sector']
    day = row['Day1 of the Week']
    time = row['Time1 of Occurrence']
    incident_count = row['IncidentCount']
    
    # If the Division is not in the dictionary, add it
    if division not in division_sector_day_time_dict:
        division_sector_day_time_dict[division] = {}
    
    # If the Sector is not in the Division's dictionary, add it
    if sector not in division_sector_day_time_dict[division]:
        division_sector_day_time_dict[division][sector] = {}
    
    # If the Day is not in the Sector's dictionary, add it
    if day not in division_sector_day_time_dict[division][sector]:
        division_sector_day_time_dict[division][sector][day] = {}
    
    # Add the Time and its corresponding incident count to the Day's dictionary
    division_sector_day_time_dict[division][sector][day][time] = incident_count

print(division_sector_day_time_dict)


In [ ]:
# Take the first 1000 entries as a subset
subset_df = df.head(1000)

# Define time slot bins
time_bins = {
    'Night': ('18:00', '23:59'),
    'Early Morning': ('00:00', '05:59'),
    'Morning': ('06:00', '11:59'),
    'Afternoon': ('12:00', '17:59')
}

# Group the data by Division, Sector, Day of the Week, and Time of Occurrence
grouped = subset_df.groupby(['Division', 'Sector', 'Day1 of the Week', 'Time1 of Occurrence']).size().reset_index(name='IncidentCount')

# Create a nested dictionary to store the data
division_sector_day_time_dict = {}

# Iterate through the rows of the grouped DataFrame to populate the nested dictionary
for _, row in grouped.iterrows():
    division = row['Division']
    sector = row['Sector']
    day = row['Day1 of the Week']
    time = row['Time1 of Occurrence']
    incident_count = row['IncidentCount']

    # Find the corresponding time slot for the given time
    time_slot = None
    for slot, (start_time, end_time) in time_bins.items():
        if start_time <= time <= end_time:
            time_slot = slot
            break

    if time_slot is None:
        continue  # Skip times that do not fall into a slot

    if division not in division_sector_day_time_dict:
        division_sector_day_time_dict[division] = {}

    if sector not in division_sector_day_time_dict[division]:
        division_sector_day_time_dict[division][sector] = {}

    if day not in division_sector_day_time_dict[division][sector]:
        division_sector_day_time_dict[division][sector][day] = {
            'Night': 0,
            'Early Morning': 0,
            'Morning': 0,
            'Afternoon': 0
        }

    division_sector_day_time_dict[division][sector][day][time_slot] += incident_count

# Now you have a nested dictionary with Division, Sector, Day, and Time Slot
print(division_sector_day_time_dict)
